In [6]:
#필요한 라이브러리 설치
import csv
import firebase_admin
from firebase_admin import credentials
from firebase_admin import db
from haversine import haversine
import sqlite3

In [7]:
#csv데이터를 2차원 리스트로 불러오기
f = open('cultureFestival.csv', 'r')
reader = csv.reader(f)
rows = []
for i in reader:
        rows.append(i) #파일 가져오기
f.close()

#default firebase database 설정
cred = credentials.Certificate('dbKey.json')
firebase_admin.initialize_app(cred, {
    'databaseURL' : 'https://dbproject-b1bec-default-rtdb.firebaseio.com/' 
    
})

ValueError: The default Firebase app already exists. This means you called initialize_app() more than once without providing an app name as the second argument. In most cases you only need to call initialize_app() once. But if you do want to initialize multiple apps, pass a second argument to initialize_app() to give each app a unique name.

In [8]:
#파이어베이스에서 사용자의 위도와 경도를 가져옴, rows[13]이 경도, rows[14]가 위도임
latitude_ref = db.reference('latitude')
longitude_ref = db.reference('longitude')
latitude_data = float(latitude_ref.get())
longitude_data = float(longitude_ref.get())

#문화축제지들의 위도와 경도를 새 리스트에 저장
latitudes = [] #위도
longitudes = [] #경도
for i in range(1, len(rows)): #i[0]은 csv의 헤더이기 떄문
    for j in range(len(rows[i])):
        if j == 13:
            latitudes.append(float(rows[i][j]))
        if j == 14:
            longitudes.append(float(rows[i][j]))

# 문화축제지와의 거리 차이를 distance 리스트에 저장. km단위, 소수 둘째자리까지
myLocation = (latitude_data, longitude_data)  #내 위치
distance = []
for i in range(len(latitudes)): #어차피 latitudes와 longitude의 길이는 같다.
    destination = (latitudes[i], longitudes[i])
    result = haversine(myLocation, destination, unit = 'km') #거리 계산
    distance.append(float(f"{result:.2f}"))

#각 rows의 [18]인덱스에 내 위치와의 거리인 distance를 삽입한다.
for i in range(1, len(rows)):
    rows[i][18] = distance[i-1] #i는 1부터 시작하기 때문  
    
#distance가 추가된 rows를 sqlite에 저장한다.
conn = sqlite3.connect('cultureFstv.db')
cursor = conn.cursor()
cursor.execute('DROP TABLE mytable') #db 초기화
cursor.execute('''
    CREATE TABLE IF NOT EXISTS mytable (
        festvlNm TEXT,
        opar TEXT,
        fstvStartDate Text,
        fstvEndDate Text,
        fstvlCo Text,
        mnnstNm Text,
        auspcInsttNm Text,
        suprtInsttNm Text,
        phoneNumber Text,
        humpageUrl Text,
        realteInfo Text,
        rdnmadr Text,
        lnmadr Text,
        latitude Text,
        longitude Text,
        referenceData Text,
        instt_code Text,
        instt_nm Text,
        distance Text
    )
''')
for i in range(1, len(rows)):
    cursor.execute('INSERT INTO mytable VALUES (?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?)', rows[i])
    conn.commit()
conn.close()


# SQLite 데이터베이스 연결
conn = sqlite3.connect('cultureFstv.db')
cursor = conn.cursor()

# SQLite에서 데이터 불러오기 (distance 열을 오름차순으로 정렬하여 상위 10개)
#사용자에게 필요하다고 생각되는 정보들만 불러온다.
cursor.execute('SELECT festvlNm, opar, fstvStartDate, fstvEndDate, fstvlCo, phoneNumber, humpageUrl, realteInfo, rdnmadr, lnmadr, distance FROM mytable ORDER BY distance ASC LIMIT 10')

distanceAsc = cursor.fetchall()

# Firestore 데이터베이스에 연결
db_ref = db.reference('/aroundMe')
db_ref.delete() #디렉토리 초기화

# 파이어베이스에 데이터 업로드
cnt = 0
for i in distanceAsc:
    data = {
            '축제 이름': i[0],
            '개최지': i[1],
            '시작일': i[2],
            '종료일': i[3],
            '축제 내용': i[4],
            '전화번호': i[5],
            '홈페이지 주소': i[6],
            '관련 정보': i[7],
            '도로명 주소': i[8],
            '지번 주소': i[9],
            '거리': i[10]
        }
    db_ref.child(str(cnt)).set(data)
    cnt += 1
#/mycollection/0이 가장 가까운 문화축제지이며
#/mycollection/9가 10번째로 가까운 문화축제지임
# 연결 종료

conn.close()
